In [1]:
REFERENCE = 'Reference'
PROJECT = 'Project'

%load_ext tensorboard

from model.model import *
from process_func import *
import process_func_ref as ref
import os
DATA_DIR = f'datasets/'
map = {'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Major': 'Military', 
       'Col': 'Military', 'Dr' : 'Other', 'Rev' : 'Other', 'Capt': 'Military', 
       'Jonkheer': 'Royal', 'Sir': 'Royal', 'Lady': 'Royal', 
       'Don': 'Royal', 'Countess': 'Royal', 'Dona': 'Royal'}
titles =['Miss', 'Mr', 'Mrs', 'Royal', 'Other', 'Master', 'Military']

# Run for validation

In [2]:
prepath = 'preprocessed'
try:
    if not os.path.exists(prepath):
        os.makedirs(prepath)
except OSError:
    print('Error Creating Directory...')
    
data = Titanic(DATA_DIR, show_head = False)
data.Preprocess(map, titles, PROJECT)
data._data.to_csv(prepath + f"/encoded_{PROJECT}.csv", index = False)
dataset = data.GetXandY()

data_ref = ref.Titanic(DATA_DIR, show_head = False)
data_ref.Preprocess(map, titles, REFERENCE)
data_ref._data.to_csv(prepath + f"/encoded_{REFERENCE}.csv", index = False)
dataset_ref = data_ref.GetXandY()

# Run for validation

Initializing...
Data Loaded.
Data Preprocessing...


c:\Users\deep\Documents\GitHub\MLBinaryClassification\process_func.py:90: RuntimeWarning: invalid value encountered in double_scalars
  cnt /= len(group_df.loc[group_df['Type'] != 'test'])


['Survived', 'Pclass', 'Sex', 'Type', 'Title', 'Family_Size', 'Baby', 'Elder', 'Family_Survival']
Done Preprocessing.
Returned Data Dictionary
Initializing...
Data Loaded.
Data Preprocessing...
Done Preprocessing.
Returned Data Dictionary


In [3]:
ref_model = ReferenceModel(dataset_ref['x_train'].shape)
CP_dir_ref = SetCheckpoint(REFERENCE)
TB_dir_ref = SetLog(REFERENCE)

Model: "ReferenceModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 16)]              0         
                                                                 
 Dense0 (Dense)              (None, 16)                272       
                                                                 
 Dense1 (Dense)              (None, 8)                 136       
                                                                 
 Output (Dense)              (None, 1)                 9         
                                                                 
Total params: 417
Trainable params: 417
Non-trainable params: 0
_________________________________________________________________


In [4]:
proj_model = ProjectModel(dataset['x_train'].shape)
CP_dir_proj = SetCheckpoint(PROJECT)
TB_dir_proj = SetLog(PROJECT)
# Run for validation

Model: "ProjectModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 19)]              0         
                                                                 
 Dense0 (Dense)              (None, 16)                320       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 batch_normalization (BatchN  (None, 16)               64        
 ormalization)                                                   
                                                                 
 Dense1 (Dense)              (None, 16)                272       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                      

In [5]:
BATCH_SIZE = 128
EPOCHS = 300
callbacks_ref = DefCallbacks(REFERENCE, CP = CP_dir_ref, TB = TB_dir_ref)
history_ref = ref_model.fit(dataset_ref['x_train'], dataset_ref['y_train'], callbacks = callbacks_ref, 
          validation_split = 0.2, batch_size=BATCH_SIZE, epochs = EPOCHS, verbose = 0)

In [6]:
BATCH_SIZE = 128
EPOCHS = 300
callbacks_proj = DefCallbacks(PROJECT, CP = CP_dir_proj, TB = TB_dir_proj)
history_proj = proj_model.fit(dataset['x_train'], dataset['y_train'], callbacks = callbacks_proj, 
          validation_split = 0.2, batch_size=BATCH_SIZE, epochs = EPOCHS, verbose = 0)

In [7]:
submission = PerformanceCheck(ref_model, CP_dir_ref, dataset_ref['x_test'], DATA_DIR)
submission.to_csv(f"submission/submission_{REFERENCE}.csv", index = False)
submission = PerformanceCheck(proj_model, CP_dir_proj, dataset['x_test'], DATA_DIR)
submission.to_csv(f"submission/submission_{PROJECT}.csv", index = False)

3/3 [==============================] - 0s 1ms/step
0    0.106543
1    0.785456
2    0.132468
3    0.160796
4    0.961181
5    0.119287
6    0.732761
7    0.102865
8    0.577484
9    0.045303
dtype: float32
ReferenceModel_Accuracy :  79.90 %
3/3 [==============================] - 0s 2ms/step
0    0.154492
1    0.431652
2    0.120504
3    0.154492
4    0.535362
5    0.154492
6    0.497960
7    0.127375
8    0.634350
9    0.131153
dtype: float32
ProjectModel_Accuracy :  79.19 %
